In [3]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

---

### Оглавление 

- Описание задачи

- Цели:
    1. Посчитайте, сколько книг вышло после 1 января 2000 года;
    2. Для каждой книги посчитайте количество обзоров и среднюю оценку;
        - 2.1 Сортировка по рейтингу:
        - 2.2 Отдельно сортировка по рейтингу:
    3. Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
    4. Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
    5. Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.


---

### Описание задачи:

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

---

### Цели:


- Посчитайть, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайть количество обзоров и среднюю оценку;
- Определить издательство, которое выпустило наибольшее число книг толще 50 страниц (таким образом из анализа исключаются брошюры);
- Определить автора с самой высокой средней оценкой книг (учитываются только книги с 50 и более оценками);
- Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

---

### 1. Посчитайте, сколько книг вышло после 1 января 2000 года;

In [4]:
query = '''
        SELECT COUNT(book_id)
        FROM books
            WHERE publication_date > '01-01-2000'
        '''

pd.io.sql.read_sql(query, con = engine)

,count
0,819


В нашем сервисе 819 книг, которые вышли после 1-ого января 2000 года.

---

### 2. Для каждой книги посчитайте количество обзоров и среднюю оценку;

#### 2.1 Сортировка по рейтингу:

In [5]:
# # INITIAL
# query = '''
#         SELECT b.book_id, COUNT(r.review_id) AS num_reviews, AVG(rt.rating) AS avg_rating
#         FROM books AS b
#         LEFT JOIN reviews AS r ON b.book_id = r.book_id
#         LEFT JOIN ratings AS rt ON b.book_id = rt.book_id
#         GROUP BY b.book_id
#         ORDER BY avg_rating
#         '''

# # pd.io.sql.read_sql(query, con = engine)

Самый высокий рейтинг у книг = 5.0, и в нашем сервисе несколько таких книг. Самый низкий рейтинг = 1.5, такой рейтинг только у одной книги в нашем списке. 

Смотрим сколько значений в reviews:

In [6]:
query = '''
SELECT book_id, COUNT(*) AS review_count 
FROM reviews 
GROUP BY book_id
ORDER BY review_count;
'''
# pd.io.sql.read_sql(query, con = engine)

В таблице с обзорами 994 значения, это значит, что не все 1000 книг имеют хотя бы один обзор.

Смотрим сколько значений в ratings:

In [7]:
query = '''
SELECT book_id, AVG(rating) AS avg_rating 
FROM ratings 
GROUP BY book_id;
'''
# pd.io.sql.read_sql(query, con = engine)

В таблице с рейтингами 1000 значений.

---

Финальный запрос, если мы учитываем книги с отсутствием значений в графе рейтинга:

In [8]:
query = '''
SELECT b.book_id, 
    COALESCE(r.avg_rating, 0) as avg_rating, 
    COALESCE(rv.num_reviews, 0) as num_reviews
    
FROM books b

LEFT JOIN (
    SELECT book_id, AVG(rating) as avg_rating
    FROM ratings
    GROUP BY book_id
) r ON b.book_id = r.book_id

LEFT JOIN (
    SELECT book_id, COUNT(*) as num_reviews
    FROM reviews
    GROUP BY book_id
) rv ON b.book_id = rv.book_id

ORDER BY num_reviews
'''
pd.io.sql.read_sql(query, con = engine)

,book_id,avg_rating,num_reviews
0,221,4.000000,0
1,83,3.666667,0
2,387,4.000000,0
3,191,4.000000,0
4,672,5.000000,0
...,...,...,...
995,627,3.789474,6
996,734,4.206897,6
997,207,3.395833,6
998,673,3.825581,6


Мы видим, что в нашей таблице по прежнему 1000 значений. 

Минимальное количество обзоров = 0, а максимальное = 7.

---

Финальный запрос, если мы отбрасываем NaN значения:

In [9]:
query = '''
SELECT b.book_id, 
    r.avg_rating as avg_rating, 
    rv.num_reviews as num_reviews
    
FROM books b

INNER JOIN (
    SELECT book_id, AVG(rating) as avg_rating
    FROM ratings
    GROUP BY book_id
) r ON b.book_id = r.book_id

INNER JOIN (
    SELECT book_id, COUNT(*) as num_reviews
    FROM reviews
    GROUP BY book_id
) rv ON b.book_id = rv.book_id

ORDER BY num_reviews

'''
pd.io.sql.read_sql(query, con = engine)

,book_id,avg_rating,num_reviews
0,413,4.000000,1
1,666,3.500000,1
2,330,5.000000,1
3,318,2.500000,1
4,639,3.500000,1
...,...,...,...
989,299,4.287500,6
990,673,3.825581,6
991,497,4.125000,6
992,750,4.125000,6


В этом случае самое высокое количество обзоров равно 7, а самое маленькое - 1.


#### 2.2 Отдельно сортировка по рейтингу:


In [10]:
query = '''
SELECT b.book_id, 
    r.avg_rating as avg_rating, 
    rv.num_reviews as num_reviews
    
FROM books b

INNER JOIN (
    SELECT book_id, AVG(rating) as avg_rating
    FROM ratings
    GROUP BY book_id
) r ON b.book_id = r.book_id

INNER JOIN (
    SELECT book_id, COUNT(*) as num_reviews
    FROM reviews
    GROUP BY book_id
) rv ON b.book_id = rv.book_id

ORDER BY num_reviews

'''
pd.io.sql.read_sql(query, con = engine)

,book_id,avg_rating,num_reviews
0,413,4.000000,1
1,666,3.500000,1
2,330,5.000000,1
3,318,2.500000,1
4,639,3.500000,1
...,...,...,...
989,299,4.287500,6
990,673,3.825581,6
991,497,4.125000,6
992,750,4.125000,6


Мы видим, что если не учитывать значения NaN, то в иотге получается меньше значений в таблице (994 вместо 1000), но по прежнему самый высокий рейтинг у книг = 5.0 и такая книга не одна, но вот самый низкий рейтинг, равный 1.5 - только у одной книги.

---

### 3. Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;

In [11]:
query = '''  
    SELECT p.publisher, COUNT(b.book_id) AS num_books
    FROM books AS b
    JOIN publishers AS p ON p.publisher_id = b.publisher_id
        WHERE b.num_pages > 50
    GROUP BY p.publisher_id
    ORDER BY num_books DESC
    LIMIT 1;
'''

pd.io.sql.read_sql(query, con = engine)

,publisher,num_books
0,Penguin Books,42


Самое активное издательство в нашем сервисе - `Penguin Books`. Оно выпустило 42 книги.

---

### 4. Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;

In [12]:
query = ''' 
            SELECT a.author, AVG(r.rating) AS avg_rating
            FROM books AS b
            LEFT JOIN authors AS a ON b.author_id = a.author_id
            LEFT JOIN ratings AS r ON b.book_id = r.book_id
                WHERE b.book_id IN (
                    SELECT book_id
                    FROM ratings
                    GROUP BY book_id
                    HAVING COUNT(*) >= 50
                )
            GROUP BY a.author
            ORDER BY avg_rating DESC
            LIMIT 1;
        '''

pd.io.sql.read_sql(query, con = engine)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.287097


Автор с самым высоким рейтингом знаменитая Джоан Роулинг. 

---

### 5. Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [13]:
query = '''
            SELECT AVG(num_reviews) AS avg_reviews
            FROM (
                  SELECT r.username, COUNT(rv.review_id) AS num_reviews
                  FROM ratings AS r
                  LEFT JOIN reviews AS rv ON r.book_id = rv.book_id AND r.username = rv.username
                  GROUP BY r.username
                  HAVING COUNT(*) > 50
            ) AS subquery
        '''

pd.io.sql.read_sql(query, con = engine)

,avg_reviews
0,24.333333


В среднем активные пользователи, которые поставили более 50 ценок, пишут чуть более 24 обзоров. 